## Erste Datenübersicht

In [ ]:
# Link to github repo:

import os
import json
import datetime
import csv

import pandas as pd
import gzip
from tqdm.autonotebook import tqdm

'''
%load_ext lab_black
%matplotlib inline
'''
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

from data_io import File_IO, Database_IO
from data_wrangling import  Data_Wrangling
from data_summarization import Data_Summarization
from setup_config import Setup_Config

## Setup

In [ ]:
setup = Setup_Config('config.ini') # loads a setup file with variables in .ini format
# the .ini file contains e.g. database connections and other settings.
#### ATTENTION: make sure to add this file to successfully to .gitignore to make sure it dont become pushed to the public repo.

file_io = File_IO() # handles input and output operations

data_wrangling = Data_Wrangling(data_io=file_io) # includes all the transformations
data_summarizations = Data_Summarization(data_io=file_io)

## Manuelle Analyse

### Auffälligkeiten


In dem Datensatz ist für jeden Patienten ein Ordner angelegt.
Jeder Patientenordner beeinhaltet einen Ordner für die unterschiedlichen Upload-Arten.
Zu jeder Upload-Art sind in dem Ordner die einzellnen Upload zu finden, z.B. als zip-Files, aber auch Excel-Files, andere Format sind möglich.

Bsp.:
Patient 73295423 enthält die Upload-Art direct-sharing-133 und darutern eine xslx Datei mit einem Suggermate-Report.


- Frage 1: Welche Upload Arten gibt es ?
- Frage 2: Welche Datentypen und Struktur ist dort erkennbar?

- Ziel: Überführung in eine einheitliche Datenstruktur.

In [ ]:
path = '/Users/me/Desktop/Code/VS-Projects/charite/OpenHumansDataTools/data/raw/n=101_OPENonOH_07.07.2022'
folder_structure = pd.DataFrame(file_io.get_folder_structure(path))
folder_structure

In [ ]:
lv1_unique = list(folder_structure.level_1.unique())
lv1_unique

#### Zu Frage 1:

Es gibt drei verschiedene Ordnerarten für die Uploads:
- direct-sharing-31
- direct-sharing-396
- direct-sharing-133

### Erste Analyse von `direct-sharing-31`

In [ ]:
file_names_lv1 = list(folder_structure[folder_structure['level_1'] == lv1_unique[0]]['file'])
file_names_lv1

Manuelle Betrachtung von `direct-sharing-31`:

Enthalten sind mit gz komprimierte json-files. 
Der Filename besteht jeweils aus einem dem Inhalt entsprechenden Text-Identifier, optional gefolgt von einer Zeitspanne. Ein Beispiel Filename sieht so aus:

`profile_2018-06-01_to_2018-06-26.json.gz`

Alternativ kann auch das Startdatum fehlen, z.B. bei:

`treatments__to_2017-10-13.json.gz`

oder gar keine Datumsrange angegeben sein, z.B. bei:

`profile.json.gz`

In [ ]:
# Frage 1.2. Welche Filenamen können auftreten?
all_names = []
for file_name in file_names_lv1:
    name = file_name.split('.')[0] # Endungen ignorieren
    name = name.split('_')[0] # if date ranges are specified, we parse them out
    all_names.append(name)

unique_file_typs = list(set(all_names))
unique_file_typs

Es gibt 4 verschiedene Arten von Files in `direct-sharing-31`:

- profile
- treatments
- devicestatus
- entries

### Erste Analyse von `direct-sharing-396`

In [ ]:
file_names_lv1 = list(folder_structure[folder_structure['level_1'] == lv1_unique[1]]['file'])
#print(lv1_unique[1])
for name in file_names_lv1: print(name)

In [ ]:
lv1 = folder_structure[folder_structure['level_1'] == lv1_unique[1]]
#lv1[~lv1['file'].str.contains('upload')]

**Manulle Analyse:**

Zip folder in der Struktur:
- upload-num82-ver1-date20201220T122408-appid6aa9e9a384264323b3c82447483b92bb.zip

Zusätzlich können auch JSON-Files enthalten sein. Die JSON-Files sind nur vorhanden, da einige Zip-Folder vorab schon manuell entpackt wurden.
Es reicht die Betrachtung der einzellnen upload-zip-Folder.

- TemporaryTargets.json
- Carbs.json
- UploadInfo.json
- VersionChanges.json
- BgReadings.json
- Preferences.json
- DisplayInfo.json
- TemporaryBasals.json
- DeviceInfo.json
- BolusCalculatorResults.json
- Boluses.json
- GlucoseValues.json
- TherapyEvents.json
- ApplicationInfo.json

In [ ]:
different_filenames = []

for name in file_names_lv1:

    upload_name_info = name.split('-')
    if not len(upload_name_info) == 5: # get exceptions from the usual structure
        different_filenames.append(name)


list(set(different_filenames)) # unique and different filenames

Relevante Frage: Was ist den upload zip-Files enthalten?

Entpacken und betrachten aller Files auf Grund von lokalen Speicherbegrenzeungen nicht möglich. Vermutung (überprüft mit Stichproben): verschiedene JSON-Files, welche die Daten entahlten.

### Erste Analyse von `direct-sharing-133`

In [ ]:
s133_file_names_lv1 = list(folder_structure[folder_structure['level_1'] == lv1_unique[2]]['file'])
#print(lv1_unique[2])
for name in s133_file_names_lv1: print(name)

In [ ]:
folder_structure[folder_structure['file'].isin(s133_file_names_lv1)]

### Manuelle Analyse

`direct-sharing-133` wird nur bei zwei Patienten verwenden. Daten sind bei einem Patienten mit Diagrammen als Analyse in einer Exceldatei zusammengefasst, die automatisch generiert wurde.
Die Daten in dem `nightscout-db.zip` file sind noch unklar und werden vorerst nicht betrachtet.

### Manuelle Analyse von Level 2

In [ ]:
folder_structure[~folder_structure['level_2'].isna()]

In Level 2 sind nur Folder enthalten, welche vermutlich zur exploaration vor dem Upload in Google Drive versehentlich entpackt wurden. Hierbei handelt es sich um in ursprünglich in Level 1 direct-sharing-396 komprimierte upload Ordner der Stuktur s.o..

Level 2 ist somit vorerst nicht relevant und wird im folgenden ignoriert.

In [ ]:
folder_structure.level_2.unique()

## Konzeptentwicklung für das Preprocessing (Normalisieren und Sammlen)

### 1. Was soll Preprocessed werden ?

Es werden in jedem Patientenordner, falls vorhanden, die folgenden Unterordner aufbereitet:
- direct-sharing-31 (ds31)
- direct-sharing-396 (ds396)

In ds31 können bis zu 4 verschieden JSON-Files vorkommen, im Namen wird ggf. mit _ getrennt eine Zeitintervall für die erhobenen Daten, ggf. auch offen, angegeben.

In ds396 werden nur die mit upload_ bezeichneten zip Folder betrachtet und zunächst davon ausgegangen, dass die relevanten Daten in json kodierten Files vorhanden sind. Alle vorkommenden JSON Files sollen vorverarbeitet werden.

### 2. Wie wird das umgesetzt ?

Für jeden Patienten soll final eine csv-Datei entstehen, in welcher sämtliche Daten aus ds31 und ds396 übersichtlich gespeichert werden. Alle csv Datein sollen zusätzlicher in einer Datnbanktabelle gespeichert werden.

Um die jeweiligen JSON-Datein in eine csv-Datei zu integrieren soll die verschachtelte Struktur der Datein aufgelöst werden, sodass diese durch Pfade der folgende Art x.y.z... ersetzt werden.
Die Pfade werden in lowercase kodiert.
Zuerst enthält der Pfad die Upload-Art, z.B. ds31. Anschließend folgt mit y der Filename und z die punktierte Auflistung des verschatelten Pfades in der JSON Datei entland den keys des dicts bis zum aktuellen Variablen Namen.

Die Ziel CSV-Datei soll die folgende Struktur enthalten: 

| patient_id         | path      | #upload| value | value_str | starttime | duration | isValid | 
|-------------------|------------|----|--|-------------|--------|---------|------------|
| 59061279 | ds31.profile.defaultprofile   | |    | Default   | 2019-07-14 05:20:22 | None |False   | 
| 59061279| ds396.profile.value1     | |2.0   |           | 2019-07-14 05:20:22 | None |True   | 


Hierdurch wird die agile Möglichkeit gegeben numerische Daten in `value` festzuhalten, alternativ textuelle Daten in `value_str`. Erste Einblicke haben gezeigt, dass einige mit Zeit angegebene Werte auch eine `duration` beeinahlten, ggf. muss diese berechnet werden.
Wenn Parameter die Gültigkeit einer Messung spezifizieren, wird diese zusätzlich mit angegeben.
Die #upload (Upload Number) gibt an, der wievielte Upload die Information enthält.
Die Struktur der csv-Datei ermöglicht das einfache filtern der Daten.

Zusätzlich wird eine Datenbank auf einem Server aufgesetzt und die finalen csv-Datein werden in eine geeignete Datenbank-Tabelle eingelesen. Für die Tabelle wird ein Index erstellt und ggf. die Datenbank Performance optimiert.
Erwartet wird eine Datengröße von bis zu 5GB pro Patient. Die Gesammtgröße von möglicheriweise > 500GB für nur 100 Patienten ist somit für Researcher schwer ohne entsprechende Infrastruktur zu managen .

Durch eine Materialized-View soll ein Überblick zu möglichen Pfaden, sowie trivialen Statistiken (wie mean, min, max) für die numerischen Variablen erfolgen.

### Welche Risiken gibt es bei der Umsetzung ?

- Es muss mit unterschiedlichen Zeitzonen gerechnet werden und die zugehörige Zeitzoneninformation in den Daten erkannt werden.

- Die Werte sind ggf. nicht mit Identifier-Value oder zeitlicher Notierung starttime angegeben.
Hier müssen mögliche Bezeichnungen gefiltert und normalisiert werden.

- Erste Einblicke haben gezeigt, dass tw. auch ein startdate und timeasseconds in Unix-Miliseconds-Time angegeben ist. Hieraus kann dann eine starttime berechnet werden.

Weitere Risiken sind denkbar.

## Erweitertes Preprocessing

Ärzten und weitere Wissenschaftlern wird die Variablenübersicht zur Verfügung gestellt.
Auf Anfrage können dann aus relevanten Variablen speziell strukturierte Materialized-Views mit Zieldaten erstellt werden.

Denkbar sind auch weitere Aufbereitungen, wie Data-Cleaning (z.B. Outlier Filtern), Datenaggregation und Umstrukturierung.